In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import accuracy_score

In [5]:
data = pd.read_csv('bank-full.csv', sep=';')

In [6]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
data.shape

(45211, 17)

In [8]:
#data['y'].value_counts()
data['pdays'].value_counts()

pdays
-1      36954
 182      167
 92       147
 91       126
 183      126
        ...  
 449        1
 452        1
 648        1
 595        1
 530        1
Name: count, Length: 559, dtype: int64

In [9]:
data_baru=data.replace({'job':{'admin.': 0, 'blue-collar': 1, 'entrepreneur': 2, 'housemaid': 3, 'management': 4,
'retired': 5, 'self-employed': 6, 'services': 7, 'student': 8, 'technician': 9,
'unemployed': 10, 'unknown': 11},
       'marital':{'divorced': 0, 'married': 1, 'single': 2},
       'education':{'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3},
       'loan':{'no':0, 'yes':1},'y':{'no':0,'yes':1}})

In [10]:
 labelencoder = LabelEncoder()
# data['job'] = labelencoder.fit_transform(data['job'])
# data['marital'] = labelencoder.fit_transform(data['marital'])
# data['education'] = labelencoder.fit_transform(data['education'])
 data_baru['default'] = labelencoder.fit_transform(data['default'])
 data_baru['housing'] = labelencoder.fit_transform(data['housing'])
# data['loan'] = labelencoder.fit_transform(data['loan'])
 data_baru['contact'] = labelencoder.fit_transform(data['contact'])
 data_baru['month'] = labelencoder.fit_transform(data['month'])
 data_baru['poutcome'] = labelencoder.fit_transform(data['poutcome'])
# data['y'] = labelencoder.fit_transform(data['y'])

In [11]:
data_baru.head()
data_baru.to_csv("data_baru.csv", index=False)

In [32]:
db= pd.read_csv('data_baru.csv', sep=',')
db

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,0


In [12]:
from scipy.stats import pearsonr, spearmanr

# Drop columns that are not needed for correlation analysis
data_baru = data_baru.drop(['duration'], axis=1)

# Calculate Pearson correlation and p-value for all numeric columns
pearson_corr = data_baru.corr(method='pearson')
pearson_pval = data_baru.corr(method=lambda x, y: pearsonr(x, y)[1])[['y']][:-1]

In [13]:
pearson_corr

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y
age,1.000000,-0.021868,-0.403240,-0.106807,-0.017879,0.097783,-0.185513,-0.015655,0.026221,-0.009120,-0.042357,0.004760,-0.023758,0.001288,0.007367,0.025155
job,-0.021868,1.000000,0.062045,0.166707,-0.006853,0.018232,-0.125363,-0.033004,-0.082063,0.022856,-0.092870,0.006839,-0.024455,-0.000911,0.011010,0.040438
marital,-0.403240,0.062045,1.000000,0.108576,-0.007023,0.002122,-0.016096,-0.046893,-0.039201,-0.005261,-0.006991,-0.008994,0.019172,0.014973,-0.016850,0.045588
education,-0.106807,0.166707,0.108576,1.000000,-0.010718,0.064514,-0.090790,-0.048574,-0.110928,0.022671,-0.057304,0.006255,0.000052,0.017570,-0.019361,0.066241
default,-0.017879,-0.006853,-0.007023,-0.010718,1.000000,-0.066745,-0.006025,0.077234,0.015404,0.009424,0.011486,0.016822,-0.029979,-0.018329,0.034898,-0.022419
balance,0.097783,0.018232,0.002122,0.064514,-0.066745,1.000000,-0.068768,-0.084350,-0.027273,0.004503,0.019777,-0.014578,0.003435,0.016674,-0.020967,0.052838
housing,-0.185513,-0.125363,-0.016096,-0.090790,-0.006025,-0.068768,1.000000,0.041323,0.188123,-0.027982,0.271481,-0.023599,0.124178,0.037076,-0.099971,-0.139173
loan,-0.015655,-0.033004,-0.046893,-0.048574,0.077234,-0.084350,0.041323,1.000000,-0.010873,0.011370,0.022145,0.009980,-0.022754,-0.011043,0.015458,-0.068185
contact,0.026221,-0.082063,-0.039201,-0.110928,0.015404,-0.027273,0.188123,-0.010873,1.000000,-0.027936,0.361145,0.019614,-0.244816,-0.147811,0.272214,-0.148395
day,-0.009120,0.022856,-0.005261,0.022671,0.009424,0.004503,-0.027982,0.011370,-0.027936,1.000000,-0.006028,0.162490,-0.093044,-0.051710,0.083460,-0.028348


In [14]:
x1 = data_baru[['age', 'job', 'marital', 'education', 'balance', 'pdays', 'previous', 'loan']]
x = data_baru[['age', 'job', 'marital', 'education', 'balance', 'pdays', 'previous', 'loan']]
y = data_baru['y']

In [15]:
scaler = StandardScaler()
scaler.fit(x)
stadarized_data = scaler.transform(x)

In [16]:
x=stadarized_data
y=data_baru['y']

In [17]:
print(x)
print(y)

[[ 1.60696496 -0.10381968 -0.27576178 ... -0.41145311 -0.25194037
  -0.43680347]
 [ 0.28852927  1.42400783  1.3683719  ... -0.41145311 -0.25194037
  -0.43680347]
 [-0.74738448 -0.71495069 -0.27576178 ... -0.41145311 -0.25194037
   2.2893591 ]
 ...
 [ 2.92540065  0.20174582 -0.27576178 ...  1.43618859  1.05047333
  -0.43680347]
 [ 1.51279098 -1.02051619 -0.27576178 ... -0.41145311 -0.25194037
  -0.43680347]
 [-0.37068857 -0.71495069 -0.27576178 ...  1.4761376   4.52357654
  -0.43680347]]
0        0
1        0
2        0
3        0
4        0
        ..
45206    1
45207    1
45208    1
45209    0
45210    0
Name: y, Length: 45211, dtype: int64


In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [19]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=10, max_features=5, random_state=45)

In [20]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features=5, random_state=45)

In [21]:
test_pred = clf.predict(X_train)
train_acc = accuracy_score(test_pred, y_train)

In [22]:
print ('Akurasi data training = ', train_acc)

Akurasi data training =  0.9043076752930768


In [23]:
y_pred = clf.predict(X_test)
test_acc = accuracy_score(y_pred, y_test)

In [24]:
print ('Akurasi data testing = ', test_acc)

Akurasi data testing =  0.8858785801172178


In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[7891,   94],
       [ 938,  120]], dtype=int64)

In [25]:
x1

,age,job,marital,education,balance,pdays,previous,loan
0,58,4,1,2,2143,-1,0,0
1,44,9,2,1,29,-1,0,0
2,33,2,1,1,2,-1,0,1
3,47,1,1,3,1506,-1,0,0
4,33,11,2,3,1,-1,0,0
...,...,...,...,...,...,...,...,...
45206,51,9,1,2,825,-1,0,0
45207,71,5,0,0,1729,-1,0,0
45208,72,5,1,1,5715,184,3,0
45209,57,1,1,1,668,-1,0,0


In [26]:
# Menampilkan feature importance dari setiap variabel independen
feature_importances = pd.DataFrame(clf.feature_importances_, index=x1.columns, columns=['Importance'])
feature_importances.sort_values('Importance', ascending=False, inplace=True)
print(feature_importances)

           Importance
pdays        0.327772
age          0.235433
balance      0.212605
job          0.069188
previous     0.065754
education    0.038213
marital      0.032504
loan         0.018532


In [27]:
# # Define LabelEncoder for each categorical column
# job_encoder = LabelEncoder()
# marital_encoder = LabelEncoder()
# education_encoder = LabelEncoder()
# default_encoder = LabelEncoder()

# # Transform categorical data into numerical data
# input_data = (58, 'management', 'married', 'tertiary', 2143, -1, 0, 'no')
# input_data_encoded = (
#     input_data[0],
#     job_encoder.fit_transform([input_data[1]])[0],
#     marital_encoder.fit_transform([input_data[2]])[0],
#     education_encoder.fit_transform([input_data[3]])[0],
#     input_data[4],
#     input_data[5],
#     input_data[6],
#     default_encoder.fit_transform([input_data[7]])[0]
# )

# # Reshape input data and standardize it
# input_data_as_numpy_array = np.array(input_data_encoded)
# input_data_reshape = input_data_as_numpy_array.reshape(1,-1)
# std_data = scaler.transform(input_data_reshape)
# print(std_data)

# # Predict using the trained classifier
# prediction = clf.predict(std_data)
# if (prediction[0]==0):
#     print ('Client not subscribed')
# else:
#     print ('Client subscribed')

# data_baru=data.replace({'job':{'admin.': 0, 'blue-collar': 1, 'entrepreneur': 2, 'housemaid': 3, 'management': 4,
# 'retired': 5, 'self-employed': 6, 'services': 7, 'student': 8, 'technician': 9,
# 'unemployed': 10, 'unknown': 11},
#        'marital':{'divorced': 0, 'married': 1, 'single': 2},
#        'education':{'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3},
#        'loan':{'no':0, 'yes':1},'y':{'no':0,'yes':1}})

input_data = (60, 5, 0, 1, 545, -1, 0, 0)
input_data_as_numpy_array = np.array(input_data)

input_data_reshape = input_data_as_numpy_array.reshape(1,-1)
std_data = scaler.transform(input_data_reshape)
print(std_data)

prediction=clf.predict(std_data)
print(prediction)

if (prediction[0]==0):
    print ('Client not subscribe')
else :
    print ('Client subscribe')

[[ 1.79531291  0.20174582 -1.91989546 -0.30055587 -0.26842166 -0.41145311
  -0.25194037 -0.43680347]]
[0]
Client not subscribe


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [28]:
import pickle

In [29]:
filename='bank_data_model.sav'
pickle.dump(clf,open(filename,'wb'))